In [ ]:
import cv2
import copy
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
from torchvision.transforms import functional as F
import os
from src.tools.utils import write_json, clear_file, is_video_detect, find_next, find_reference
from src.tools.VideoClip import VideoClip
from src.models.PoseDetect import PoseDetect
from src.models.CourtDetect import CourtDetect
from src.models.NetDetect import NetDetect
import argparse
from src.tools.BallDetect import ball_detect
import warnings
# clear the polyfit Rankwarning
warnings.simplefilter('ignore', np.RankWarning)

result_path="C:/Users/86153/Desktop/res"
name_list=[
    "Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals",
    "Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_BWF_World_Championships_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_England_Open_2022_Semi_finals",
    "An_Se_Young_Chen_Yu_Fei_PERODUA_Malaysia_Masters_2022_Finals",
    "AN_Se_Young_Gregoria_Mariska_TUNJUNG_Malaysia_Masters_2022_SemiFinals",
    "AN_Se_Young_Pornpawee_CHOCHUWONG_Korea_Open_Badminton_Championships_2022_Finals",
    "AN_Seyoung_PUSARLA_V",
    "Anders_Antonsen_Viktor_Axelsen_HSBC_BWF_WORLD_TOUR_FINALS_2020_Finals",
    "Anthony_Sinisuka_Ginting_Rasmus_Gemke_YONEX_Thailand_Open_2021_QuarterFinals",
]
for video_name in name_list:
    print("-" * 10 + "Starting Ball Detection" + "-" * 10)       
    for res_root, res_dirs, res_files in os.walk(
            f"{result_path}/videos/{video_name}"):
        for res_file in res_files:
            _, ext = os.path.splitext(res_file)
            if ext.lower() in ['.mp4']:
                res_video_path = os.path.join(res_root, res_file)
                print(res_video_path)
                ball_detect(res_video_path, f"{result_path}/ball")
    print("-" * 10 + "End Badminton Detection" + "-" * 10)


In [1]:
import torch

print(torch.__version__)

2.0.1+cu117


In [23]:
import cv2
import copy
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
from torchvision.transforms import functional as F
import os
from src.tools.utils import write_json, read_json,clear_file, is_video_detect, find_next, find_reference
from src.tools.VideoClip import VideoClip
from src.models.PoseDetect import PoseDetect
from src.models.CourtDetect import CourtDetect
from src.models.NetDetect import NetDetect
import argparse
from src.tools.BallDetect import ball_detect
import warnings
# clear the polyfit Rankwarning
warnings.simplefilter('ignore', np.RankWarning)

result_path="E:\\res"
name_list=[
    "Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals",
    "Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_BWF_World_Championships_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_England_Open_2022_Semi_finals",
    "An_Se_Young_Chen_Yu_Fei_PERODUA_Malaysia_Masters_2022_Finals",
    "AN_Se_Young_Gregoria_Mariska_TUNJUNG_Malaysia_Masters_2022_SemiFinals",
    "AN_Se_Young_Pornpawee_CHOCHUWONG_Korea_Open_Badminton_Championships_2022_Finals",
    "AN_Seyoung_PUSARLA_V",
    "Anders_Antonsen_Viktor_Axelsen_HSBC_BWF_WORLD_TOUR_FINALS_2020_Finals",
    "Anthony_Sinisuka_Ginting_Rasmus_Gemke_YONEX_Thailand_Open_2021_QuarterFinals",
]
print(result_path)
for video_name in name_list:
    print("-" * 10 + "Starting Ball Detection" + "-" * 10)       
    json_path=os.path.join(f"{result_path}/players/player_kp",f"{video_name}.json")
    print(json_path)
    # break

    info_dict=read_json(json_path)
    player_dict={}
    # 遍历 JSON 对象中的每个键值对
    for key, value in info_dict.items():
        if value['top'] is None or value['bottom'] is None:
            continue
        player_dict[key]={
            'top':value['bottom'],
            'bottom':value['top']
        }
        print(key)

        write_json(player_dict, video_name,f"E:/ans/players/player_kp")

    print("-" * 10 + "End Badminton Detection" + "-" * 10)


E:\res
----------Starting Ball Detection----------
E:\res/players/player_kp\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals.json
18195
18196
18197
18198
18199
18200
18201
18202
18203
18204
18205
18206
18207
18208
18209
18210
18211
18212
18213
18214
18215
18216
18217
18218
18219
18220
18221
18222
18223
18224
18225
18226
18227
18228
18229
18230
18231
18232
18233
18234
18235
18236
18237
18238
18239
18240
18241
18242
18243
18244
18245
18246
18247
18248
18249
18250
18251
18252
18253
18254
18255
18256
18257
18258
18259
18260
18261
18262
18263
18264
18265
18266
18267
18268
18269
18270
18271
18272
18273
18274
18275
18276
18277
18278
18279
18280
18281
18282
18283
18284
18285
18286
18287
18288
18289
18290
18291
18292
18293
18294
18295
18297
18298
18401
18402
18403
18404
18405
18406
18407
18408
18409
18410
18411
18412
18413
18414
18415
18416
18417
18418
18419
18420
18421
18422
18423
18424
18425
18426
18427
18428
18429
18430
18431
18432
18433
18434
18435
18436
18437
18438
1843

KeyboardInterrupt: 